In [2]:
import keras_ocr
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import re

2023-07-06 06:04:25.796499: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
pipeline = keras_ocr.pipeline.Pipeline()

Looking for /Users/rsoedarnadi/.keras-ocr/craft_mlt_25k.h5


2023-07-06 06:04:40.056445: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-06 06:04:40.058608: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-06 06:04:40.061252: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Looking for /Users/rsoedarnadi/.keras-ocr/crnn_kurapan.h5


In [4]:
test_df = pd.read_csv('DataTestLabelled.csv')
list_license_plates = test_df['Image Labels']
list_file_names = test_df['Name of File']

In [5]:
full_path = []
for file in list_file_names:
    full_path.append(os.path.join(os.getcwd(),'Test-Data',file))

In [18]:
generated_plate_numbers = []
for path in full_path:
    image = [
    keras_ocr.tools.read(path)
    ]
    prediction_groups = pipeline.recognize(image)
    concat_output = "".join(row[0] for row in prediction_groups[0])
    generated_plate_numbers.append(concat_output.upper())

1/1 [==============================] - 0s 369ms/step


In [25]:
for i, platenum in enumerate(generated_plate_numbers):
   generated_plate_numbers[i] = re.sub('[^A-Za-z0-9]+', '', platenum)

In [19]:
def calculate_predicted_accuracy(names, actual_list, predicted_list):
    list_accuracy = []
    for name, actual_plate, predict_plate in zip(names, actual_list, predicted_list):
        accuracy = 0
        num_matches = 0
        if actual_plate == predict_plate:
            accuracy = 1.0
        else:
            for a, p in zip(actual_plate, predict_plate):
                if a == p:
                    num_matches += 1
            accuracy = round((num_matches / len(actual_plate)), 2)
        list_accuracy.append(accuracy)
    list_accuracy = np.asarray(list_accuracy)
    avg_value = np.average(list_accuracy)
    return avg_value

In [26]:
calculate_predicted_accuracy(list_file_names, list_license_plates, generated_plate_numbers)

0.2962